In [9]:
large_example = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

In [378]:
inp = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

In [96]:
inp = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""

In [1]:
inp = open('data15').read()

In [265]:
grid,instructions = inp.split('\n\n')
grid = grid.splitlines()
instructions = instructions.replace('\n',"")

In [1]:
# p2
def p2_grid(inp):
    grid,instructions = inp.split('\n\n')
    grid = grid.splitlines()
    instructions = instructions.replace('\n',"")
    for i in range(len(grid)):
        grid[i] = grid[i].replace("#","##")
        grid[i] = grid[i].replace("O","[]")
        grid[i] = grid[i].replace(".","..")
        grid[i] = grid[i].replace("@","@.")
    return grid, instructions
#grid, instructions = p2_grid(inp)

In [370]:
grid

['####################################################################################################',
 '##....[][]............[][][]..............##[]..##[]##....[]##..[]..[]..##....[]##..[]........[][]##',
 '##......[][]....[]....##......[]..[][]......[]......[][][][][][]##[]......##......[]....[]##..[][]##',
 '##......[][]##[]........[]..............[]..............[][]....[]....[]....[]##[][][][][]..##..####',
 '##[]........##......[]....##......##[][]..[]......##[]....[]..[]##[]..[]........[]..........[]....##',
 '##[][]....[]........[]................[][]....##[]......####......[][]..................[][][][]..##',
 '##..[]....[]..##....[][]##............[]..............[]....[]..............[]..[]..##....##....####',
 '##..................##..[][][][][]........[]......[]....[]..[][]........[]........####....##..[]..##',
 '##..[]..[]..[][]..##....[]....[]......##[]..............[][][]..[][]......##[]..##..##[]..[]......##',
 '##......##......[][]......##[][]....[]....[]##[]..[][

try to move 
if obstacle
iterate through direction to find spaces


In [164]:
boxes = set((i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == 'O')
immovable = set((i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '#')
robot_pos = [(i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '@'][0]

In [2]:
directions = {'<' : (0,-1),
              'v' : (1,0),
              '^' : (-1,0),
              '>' : (0,1)
}

In [ ]:
def move_box(dir_icon, box_pos, boxes, obstacles):

    dir_i, dir_j = directions.get(dir_icon)

    box_i, box_j = box_pos

    next_pos = (box_i + dir_i, box_j + dir_j)

    if next_pos in obstacles: # no movement
        return boxes, False # no change
    
    elif next_pos in boxes:
        new_boxes, free_space = move_box(dir_icon, next_pos, boxes, obstacles)
        if free_space:
            new_boxes.remove(box_pos)
            new_boxes.add(next_pos)
        return new_boxes, free_space

    else: #  next_pos is free
        new_boxes = boxes
        new_boxes.remove(box_pos)
        new_boxes.add(next_pos)
        return new_boxes, True

In [167]:
def move_robot(dir_icon, robot_pos, boxes, obstacles):

    dir_i, dir_j = directions.get(dir_icon)

    robot_i, robot_j = robot_pos

    if (robot_i + dir_i, robot_j + dir_j) in obstacles: # no movement
        return robot_pos, boxes
    
    n = 1

    new_robot_pos = robot_pos
    new_boxes = boxes

    while True:

        next_pos = (robot_i + (n*dir_i), robot_j + (n*dir_j))

        if (next_pos) in obstacles: # no movement
            return new_robot_pos, new_boxes
        
        elif next_pos in boxes: # hits a box
            new_boxes, free_space = move_box(dir_icon, next_pos, boxes, obstacles)
            if free_space:
                new_robot_pos = next_pos
            return new_robot_pos, new_boxes
        
        else: # free space
            return next_pos, boxes

In [168]:
robot_pos = [(i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '@'][0]
boxes = set((i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == 'O')

for dir_icon in instructions:
    robot_pos, boxes = move_robot(dir_icon, robot_pos, boxes, immovable)


recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion
recursion


In [169]:
ans = 0
for i,j in boxes:
    ans = ans + (100*i) + j

In [170]:
ans

1486930

In [149]:
robot_pos

(4, 4)

In [150]:
boxes

{(1, 5), (1, 6), (3, 6), (4, 3), (5, 4), (6, 4)}

# Part 2

In [3]:
from dataclasses import dataclass
from typing import Tuple

@dataclass(frozen=True)
class Coordinate:
    i: float
    j: float

    def as_tuple(self) -> tuple:
        """Return the coordinate as a tuple."""
        return (self.i, self.j)

    def move(self, dx: float, dy: float) -> 'Coordinate':
        """Return a new Coordinate moved by dx and dy."""
        return Coordinate(self.i + dx, self.j + dy)
    
    def __repr__(self) -> str:
        return f"({self.i},{self.j})"


@dataclass(frozen=True)
class Block:
    start: Coordinate

    @property
    def end(self) -> Coordinate:
        """Compute the end coordinate as one unit to the right of start."""
        return Coordinate(self.start.i, self.start.j+1)

    def move(self, dx: float, dy: float) -> 'Block':
        """Return a new Block moved by dx and dy."""
        new_start = self.start.move(dx, dy)
        return Block(start=new_start)
    
    def point_in(self,point: tuple):
        if point == self.start.as_tuple() or point == self.end.as_tuple():
            return True
        else:
            return False
        
    def ans(self, grid):
        i_len = len(grid)
        j_len = len(grid[0])
        min_i = min(abs(i_len - self.start.i), abs(self.start.i - i_len)) - 1
        min_j = min(abs(j_len - self.start.j), abs(self.start.j - j_len), abs(j_len-self.end.j), abs(self.end.j - j_len)) - 2
        return min_i, min_j, (100*min_i) + min_j
    
    def ans_new(self, grid):
        return (100*self.start.i) + (self.start.j)

    def __repr__(self) -> str:
        return f"Block({self.start},{self.end})"


# Example Usage
if __name__ == "__main__":
    # Initialize the block with start and end coordinates
    block = Block(start=Coordinate(1, 5))
    print(f"Initial Block: {block}")  # Output: Block(start=Coordinate(x=1, y=5), end=Coordinate(x=3, y=2))
    
    # Move the block by (2, -1)
    moved_block = block.move(0, -1)
    print(f"Moved Block: {moved_block}")  # Output: Block(start=Coordinate(x=3, y=4), end=Coordinate(x=5, y=1))
    

Initial Block: Block((1,5),(1,6))
Moved Block: Block((1,4),(1,5))


In [4]:
def check_in_blocks(point: tuple,blocks: list):
    #print()
    #print(f'checking to see if {point} is in {blocks}')
    #print()

    for b in blocks:
        if b.point_in(point):
            return b
    else:
        return False

In [5]:
def is_block_in_obstacles(block: Block, obstacles) -> bool:
    return (
        block.start.as_tuple() in obstacles or
        block.end.as_tuple() in obstacles
    )

In [6]:
def move_robot(dir_icon, robot_pos, boxes, obstacles):
    #print()
    #print(f'boxes: {boxes}')
    #print(f'moving robot: {dir_icon}')

    dir_i, dir_j = directions.get(dir_icon)

    robot_i, robot_j = robot_pos

    if (robot_i + dir_i, robot_j + dir_j) in obstacles: # no movement
        return robot_pos, boxes
    
    n = 1

    new_robot_pos = robot_pos
    new_boxes = boxes

    while True:

        next_pos = (robot_i + (n*dir_i), robot_j + (n*dir_j))
        b = check_in_blocks(next_pos, new_boxes)

        if (next_pos) in obstacles: # no movement
            return new_robot_pos, new_boxes
        
        elif b: # hits a box - see if we can move this one
            #print(f'robot hit box at {b}')
            new_boxes, free_space = move_box(dir_icon, b, boxes, obstacles)
            if free_space:
                new_robot_pos = next_pos
            return new_robot_pos, new_boxes
        
        else: # free space
            return next_pos, new_boxes

In [7]:
def move_box(dir_icon :str, box: Block, boxes: list, obstacles: list):
    #print(f'attempting to move {box}')

    dir_i, dir_j = directions.get(dir_icon)

    next_pos = box.move(dir_i, dir_j)
    #print(f'next_pos.start: {next_pos.start}')

    boxes_without_current = boxes - {box}


    bl = check_in_blocks(next_pos.start.as_tuple(), boxes_without_current) #does the left side of the box hit anything?
    br = check_in_blocks(next_pos.end.as_tuple(), boxes_without_current) # does the right side of the box hit anything?

    #print(f'bl: {bl}')
    #print(f'br: {br}')



    if is_block_in_obstacles(next_pos,obstacles): # no movement
        #print('box blocked by obstacle')
        return boxes, False # no change
    
    if bl or br:
        if bl: # if left of new position hits a box
            #print('left of new box hits box')
            new_boxes, free_space_left = move_box(dir_icon, bl, boxes, obstacles)

            if br and br != bl:
                #print('hit two different boxes!')
                new_boxes, free_space_right = move_box(dir_icon, br, new_boxes, obstacles)

                if free_space_right and free_space_left:
                    new_boxes.remove(box)
                    new_boxes.add(next_pos)
                    return new_boxes, True
                
                else:
                    return boxes, False
            
            if free_space_left:
                new_boxes.remove(box)
                new_boxes.add(next_pos)
            return new_boxes, free_space_left
        
        else: # right of new position hits box
            #print('right side hits box')
            new_boxes, free_space = move_box(dir_icon, br, boxes, obstacles)
            if free_space:
                new_boxes.remove(box)
                new_boxes.add(next_pos)

            return new_boxes, free_space

    else: #  next_pos is free
        #print(f'Block moves into free space!')
        new_boxes = boxes
        new_boxes.remove(box)
        new_boxes.add(next_pos)
        return new_boxes, True

find all connected boxes in the direction of interest

find the frontier of the boxes

In [8]:
def p2(inp):
    grid, instructions = p2_grid(inp)

    robot_pos = [(i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '@'][0]
    boxes = set(Block(start=Coordinate(i, j)) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '[')
    immovable = set((i,j) for i in range(len(grid)) for j in range(len(grid[0])) if grid[i][j] == '#')

    for dir_icon in instructions:
        robot_pos, boxes = move_robot(dir_icon, robot_pos, boxes, immovable)

    ans = 0
    for b in boxes:
        ans += b.ans_new(grid)

    return ans

In [10]:
p2(large_example)

9021

In [11]:
p2(open('data15').read())

1484674